In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder

import plotly.express as px

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
imdb_df = pd.read_csv('/content//drive/MyDrive/Masters AUA/Spring 2022/Machine Learning/Project/final_imdb.csv')
imdb_clean = pd.DataFrame()

In [3]:
imdb_df.shape

(312521, 14)

In [4]:
imdb_df.head()

,actors,director,duration,genre,imdb_rating,link,synopsis,title,votes,page_url,page_url_cleaned,release_start,release_month,tv_series
0,"['John Dall', 'Peggy Cummins', 'Berry Kroeger'...",Joseph H. Lewis,87.0,"Crime, Drama, Film-Noir",7.6,https://www.imdb.com/title/tt0042530/,Two disturbed young people release their fasci...,Gun Crazy,13192.0,https://www.imdb.com/search/title/?release_dat...,https://www.imdb.com/search/title/?release_dat...,1950.0,1,0
1,"['Raoul Walsh', 'Errol Flynn', 'Alexis Smith',...",Ray Enright,76.0,Western,6.1,https://www.imdb.com/title/tt0042744/,An Australian sheepman comes to Montana lookin...,Montana,1215.0,https://www.imdb.com/search/title/?release_dat...,https://www.imdb.com/search/title/?release_dat...,1950.0,1,0
2,"['Randolph Scott', 'Dorothy Malone', 'Forrest ...",Gordon Douglas,81.0,Western,6.3,https://www.imdb.com/title/tt0042782/,A mysterious stranger crosses paths with an ou...,The Nevadan,919.0,https://www.imdb.com/search/title/?release_dat...,https://www.imdb.com/search/title/?release_dat...,1950.0,1,0
3,"['Gene Tierney', 'Richard Conte', 'José Ferrer...",Otto Preminger,98.0,"Crime, Drama, Film-Noir",6.7,https://www.imdb.com/title/tt0042039/,A woman suffering from kleptomania is hypnotiz...,Whirlpool,4206.0,https://www.imdb.com/search/title/?release_dat...,https://www.imdb.com/search/title/?release_dat...,1950.0,1,0
4,"['Robert Preston', 'Robert Sterling', 'Chill W...",George Templeton,83.0,Western,5.6,https://www.imdb.com/title/tt0043013/,Brother is pitted against brother in this tale...,The Sundowners,480.0,https://www.imdb.com/search/title/?release_dat...,https://www.imdb.com/search/title/?release_dat...,1950.0,1,0


# Encoding

## Actors

In [5]:
# Actors
print('Unique elements', (imdb_df['actors'].nunique()))
# Calculating the number of top cast
n_actors = [len(eval(imdb_df['actors'][i])) for i in range(imdb_df.shape[0])]
imdb_df['n_actors'] = n_actors
px.histogram(imdb_df, 'n_actors')

Unique elements 287807


In [6]:
# As there are films, where the actors' tag was not correctly identified (actors instead of the directors), the number of actors is huge.
# I will drop the films which have more than 7 actors, and don't have actors at all
print(imdb_df.shape)
imdb_df = imdb_df[(imdb_df.n_actors <= 4) & (imdb_df.n_actors != 0)].reset_index(drop=True)
print(imdb_df.shape)

(312521, 15)
(277967, 15)


In [7]:
px.histogram(imdb_df, 'n_actors')

In [8]:
imdb_df['actors'] = [eval(i) for i in imdb_df['actors']]
imdb_clean[['actor1', 'actor2', 'actor3', 'actor4']] = pd.DataFrame(imdb_df['actors'].tolist())

In [9]:
for i in ['actor1', 'actor2', 'actor3', 'actor4']:
    # creating instance of labelencoder
    labelencoder = LabelEncoder()
    # Assigning numerical values and storing in another column
    imdb_clean[i] = labelencoder.fit_transform(imdb_clean[i])

In [10]:
imdb_clean.describe()

,actor1,actor2,actor3,actor4
count,277967.000000,277967.000000,277967.000000,277967.000000
mean,61195.458000,69066.077829,74389.588401,72229.946803
std,34987.187583,39413.263328,42085.399311,36240.416365
min,0.000000,0.000000,0.000000,0.000000
25%,31003.500000,35177.000000,38217.000000,40539.500000
50%,61124.000000,69016.000000,74661.000000,80436.000000
75%,91846.000000,103377.000000,111569.500000,109379.000000
max,121412.000000,131974.000000,137173.000000,109379.000000


## Director

In [11]:
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
imdb_clean['director'] = labelencoder.fit_transform(imdb_df['director'])

In [12]:
imdb_clean

,actor1,actor2,actor3,actor4,director
0,55016,97085,13731,73544,49418
1,93259,31026,39521,31712,33799
2,37704,103090,64231,16239,73749
3,96700,104953,21570,15711,32329
4,47936,29996,59786,86615,9312
...,...,...,...,...,...
277962,37786,107107,5397,2560,23853
277963,91134,113540,103311,106341,77751
277964,47563,15714,8961,9933,12494
277965,75617,35822,109649,109379,88617


## Genre

In [13]:
genres = [i.split(', ') if i == i else ['Unknown'] for i in imdb_df['genre']]
mlb = MultiLabelBinarizer()
imdb_clean[mlb.classes_] = pd.DataFrame(mlb.fit_transform(genres), columns = mlb.classes_)


In [14]:
imdb_df.columns

Index(['actors', 'director', 'duration', 'genre', 'imdb_rating', 'link',
       'synopsis', 'title', 'votes', 'page_url', 'page_url_cleaned',
       'release_start', 'release_month', 'tv_series', 'n_actors'],
      dtype='object')

## Creating final df

In [15]:
imdb_clean[['duration', 'imdb_rating', 'votes', 'release_start', 'release_month', 'tv_series', 'title', 'synopsis']] = \
    imdb_df[['duration', 'imdb_rating', 'votes', 'release_start', 'release_month', 'tv_series', 'title', 'synopsis']]

In [16]:
imdb_clean.isna().sum()

actor1               0
actor2               0
actor3               0
actor4               0
director             0
Action               0
Adult                0
Adventure            0
Animation            0
Biography            0
Comedy               0
Crime                0
Documentary          0
Drama                0
Family               0
Fantasy              0
Film-Noir            0
Game-Show            0
History              0
Horror               0
Music                0
Musical              0
Mystery              0
News                 0
Reality-TV           0
Romance              0
Sci-Fi               0
Short                0
Sport                0
Talk-Show            0
Thriller             0
Unknown              0
War                  0
Western              0
duration         49807
imdb_rating      69794
votes            69794
release_start      121
release_month        0
tv_series            0
title                1
synopsis             5
dtype: int64

# Feature Engineering

## Topic modeling

In [17]:
!pip install bertopic

In [18]:
import re
import nltk
from nltk.corpus import stopwords

import gensim
from gensim.utils import simple_preprocess

from bertopic import BERTopic

In [19]:
# deleting rows with no synopsis
imdb_clean = imdb_clean[imdb_clean['synopsis'] != 'Add a Plot'].dropna(subset=['synopsis']).reset_index(drop=True)

In [20]:
imdb_clean.shape

(196712, 42)

In [21]:

# Removing punctuation
imdb_clean['synopsis'] = imdb_clean['synopsis'].map(lambda x: re.sub('[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]', '', x))
# Converting the text to lowercase
imdb_clean['synopsis'] = imdb_clean['synopsis'].map(lambda x: x.lower())
# Removing 'see full summary'
imdb_clean['synopsis'] = imdb_clean['synopsis'].map(lambda x: re.sub('see full summary\xa0»', '', x))
# Deleting unnecessary spaces
imdb_clean['synopsis'] = imdb_clean['synopsis'].str.strip()

In [22]:
# Removing stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
             
data = imdb_clean['synopsis'].values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## BERT

In [23]:
model = BERTopic(min_topic_size=3, n_gram_range=(1,3), verbose=True)

In [24]:
docs = data_words
docs = np.array([(" ").join(i) for i in docs])
# topics = model.fit(docs)

In [25]:
labels, probs = model.fit_transform(docs)

Batches:   0%|          | 0/6148 [00:00<?, ?it/s]

2022-05-07 10:17:44,024 - BERTopic - Transformed documents to Embeddings
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning:

The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.

2022-05-07 10:23:40,213 - BERTopic - Reduced dimensionality
2022-05-07 10:24:52,591 - BERTopic - Clustered reduced embeddings


In [26]:
imdb_clean['topic'] = labels

In [29]:
imdb_clean.to_csv('/content//drive/MyDrive/Masters AUA/Spring 2022/Machine Learning/Project/imdb_clean.csv')

In [30]:
imdb_clean.to_csv('/content//drive/MyDrive/Masters AUA/Spring 2022/Machine Learning/Project/imdb_clean.csv', index=False)

In [31]:
import pickle
filename = '/content//drive/MyDrive/Masters AUA/Spring 2022/Machine Learning/Project/topic_modeling_model.sav'
pickle.dump(model, open(filename, 'wb'))

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [36]:
model.get_topic_freq()

,Topic,Count
0,-1,115959
1,0,1013
2,1,763
3,2,561
4,3,556
...,...,...
6673,7710,3
6672,7711,3
6671,7712,3
6670,7703,3


In [ ]:
model.visualize_topics()

In [27]:
# topics.visualize_topics()

In [28]:
# topics.visualize_barchart(top_n_topics=12)